# RvF - Starter Code Workbook

This is a workbook that you can upload to Google Colab to work on the project. It will download all the correct files and details to work on model development and improving.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip -q install grad-cam #gradcam

# Change this to the folder containing your Kaggle API key (kaggle.json)
%env KAGGLE_KEY_FOLDER=Colab Notebooks
!mkdir data
!export KAGGLE_CONFIG_DIR=/content/drive/MyDrive/$KAGGLE_KEY_FOLDER && wget -O - "https://raw.githubusercontent.com/MichiganDataScienceTeam/W24-RvF/main/data/download.sh" | bash -s rvf10k

Streaming output truncated to the last 5000 lines.
  inflating: data/rvf10k/rvf10k/train/real/28056.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28081.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28160.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28181.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28190.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28192.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28194.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28197.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28199.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28247.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28281.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28308.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28316.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28332.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28342.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28345.jpg  
  inflating: data/rvf10k/rvf10k/train/real/28347.jpg  
  inflating: d

In [2]:
!rm -r W24-RvF starter_code
!git clone -q https://github.com/MichiganDataScienceTeam/W24-RvF.git
!mv W24-RvF/starter_code .
!rm -r W24-RvF

rm: cannot remove 'W24-RvF': No such file or directory
rm: cannot remove 'starter_code': No such file or directory


**KEY**: Make sure to save your work after every meeting! We recommend you do so via GitHub, but its not necessary.

The following sample code is the basics you need to get started with model development.

In [3]:
import torch
import torchvision.transforms.v2 as v2
import torch
from torch import nn, optim

In [4]:
if torch.cuda.is_available():
  print("CUDA available. Using GPU acceleration.")
  device = "cuda"
else:
  print("CUDA is NOT available. Using CPU for training.")
  device = "cpu"

CUDA available. Using GPU acceleration.


In [14]:
def preprocess(image) -> torch.Tensor:
    """
    Preprocesses an image by applying a series of transformation.

    Args:
        image (npt.ArrayLike): The input image to be preprocessed.

    Returns:
        torch.Tensor: The preprocessed image as a tensor.
    """

    tensor_converter = v2.Compose([ # Step 0: Convert from PIL Image to Torch Tensor (this is always necessary)
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True) # scale=True sets it back into [0,1]
    ])
    jitter = v2.ColorJitter(brightness=.5, hue=.3) # random color shift
    hflipper = v2.RandomHorizontalFlip(p=0.5) # 50% chance to horizontal flip
    vflipper = v2.RandomVerticalFlip(p=0.5) # 50% chance to vertical flip
    cropper = v2.RandomCrop(size=(200, 200))

    preprocessor = v2.Compose([
        tensor_converter,
        jitter,
        #hflipper,
        #vflipper,
        cropper,
    ])
    return preprocessor(image)

In [15]:
import torch
from torchvision import models, transforms, datasets

#model_name =

source_model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
print(source_model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [16]:
source_model.classifier = torch.nn.Sequential(
    # 512x49
    torch.nn.Linear(25088, 1024), # 25088 x 1024
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1024, 128), # 1024 x 128
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(128, 2) # 128 x 10
)

In [17]:
print(source_model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [18]:
#for parameter in source_model.features.parameters():
    #parameter.requires_grad = False

for parameter in source_model.modules():
    parameter.requires_grad = False

In [19]:
source_model = source_model.to('cuda')

In [20]:
class Model(nn.Module):
    # TODO: Define your CNN here!
    def __init__(self):
      """Constructor for the neural network."""
      super(Model, self).__init__()        # Call superclass constructor
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1)
      self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
      self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
      self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
      self.relu = nn.ReLU()
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(2048, 512)
      self.fc2 = nn.Linear(512, 64)
      self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        z1 = self.conv1(x)
        h1 = self.relu(z1)
        z2 = self.conv2(h1)
        h2 = self.relu(z2)
        p1 = self.pool(h2)

        z3 = self.conv3(p1)
        h3 = self.relu(z3)
        z4 = self.conv4(h3)
        h4 = self.relu(z4)
        p2 = self.pool(h4)

        flat = self.flatten(p2)
        z = self.fc1(flat)
        z = self.relu(z)
        z = self.fc2(z)
        z = self.relu(z)
        z = self.fc3(z)

        return z

In [23]:
from submission.dataset import get_loaders
from submission.train import train_model, plot_performance, load_model
from submission.model import Model
import torch
import os

# TODO: To test locally, change dataset from `data/rvk140k` to whichever dataset you use locally
train_loader, val_loader = get_loaders(data_directory="data/rvf140k")

model = source_model()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
criterion = torch.nn.CrossEntropyLoss()

checkpoint_dir = f"checkpoints_{os.getenv('SLURM_JOB_ID')}"
history = train_model(
    model,
    criterion,
    optimizer,
    train_loader,
    val_loader,
    epochs=5,
    checkpoint_dir=checkpoint_dir,
)
plot_performance(history)

ModuleNotFoundError: No module named 'submission'

In [21]:
from starter_code.dataset import get_loaders
from starter_code.train import train_model, plot_performance, load_model

train_loader, val_loader = get_loaders(preprocessor=preprocess)

model = source_model # Model()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # TODO: Change the optimizer to explore different options
criterion = torch.nn.CrossEntropyLoss() # TODO: Change the criterion to explore different options

history = train_model(model, criterion, optimizer, train_loader, val_loader, epochs=20)
plot_performance(history)

# Load the model from the training run
load_model(model, "checkpoints", 1) # change epoch from 0 to something else

Batch Accuracy: 0.88 , Total Accuracy: 0.78: 100%|██████████| 219/219 [01:21<00:00,  2.69it/s]


Epoch 1: Loss - (Train 128.94/Val 0.01), Accuracy - (Train 0.78/Val 0.84)


Batch Accuracy: 0.83 , Total Accuracy: 0.89: 100%|██████████| 219/219 [01:19<00:00,  2.74it/s]


Epoch 2: Loss - (Train 59.76/Val 0.01), Accuracy - (Train 0.89/Val 0.86)


Batch Accuracy: 0.96 , Total Accuracy: 0.92: 100%|██████████| 219/219 [01:21<00:00,  2.70it/s]


...

Epoch 14: Loss - (Train 12.23/Val 0.00), Accuracy - (Train 0.98/Val 0.96)


Batch Accuracy: 1.00 , Total Accuracy: 0.98: 100%|██████████| 219/219 [01:20<00:00,  2.71it/s]


Epoch 15: Loss - (Train 9.47/Val 0.01), Accuracy - (Train 0.98/Val 0.93)


Batch Accuracy: 1.00 , Total Accuracy: 0.98:  84%|████████▍ | 185/219 [01:08<00:12,  2.69it/s]


KeyboardInterrupt: 

**KEY**: At the end of each work session, submit this workbook via slack! This will allow the project leads to train your model on the larger 140k dataset over the week!